In [124]:
import requests

crtfc_key = 'be1cf4dc3136fbd49a414efc1dd8141b30c6354f'
corp_code = '01096341'
# bgn_de =
# end_de =
page_count = max

webpages = []

bsns_year = ['2020', '2021', '2022', '2023', '2024']
for year in bsns_year:
  url = 'https://opendart.fss.or.kr/api/list.xml?crtfc_key={}&corp_code={}&pblntf_ty=A&corp_cls=K&bgn_de={}&end_de={}&%page_count=100&pblntf_detail_ty=A001&pblntf_detail_ty=A002&pblntf_detail_ty=A003'.format(crtfc_key, corp_code, year+'0101', year+'1231')


  resp = requests.get(url)

  webpages.append(resp.content.decode('UTF-8'))
  #webpages.append(web)
print(webpages)

['<?xml version="1.0" encoding="UTF-8" standalone="yes"?><result><status>000</status><message>정상</message><page_no>1</page_no><page_count>10</page_count><total_count>4</total_count><total_page>1</total_page><list><corp_code>01096341</corp_code><corp_name>넥슨게임즈</corp_name><stock_code>225570</stock_code><corp_cls>K</corp_cls><report_nm>분기보고서 (2020.09)</report_nm><rcept_no>20201111000226</rcept_no><flr_nm>넥슨게임즈</flr_nm><rcept_dt>20201111</rcept_dt><rm></rm></list><list><corp_code>01096341</corp_code><corp_name>넥슨게임즈</corp_name><stock_code>225570</stock_code><corp_cls>K</corp_cls><report_nm>반기보고서 (2020.06)</report_nm><rcept_no>20200813000531</rcept_no><flr_nm>넥슨게임즈</flr_nm><rcept_dt>20200813</rcept_dt><rm></rm></list><list><corp_code>01096341</corp_code><corp_name>넥슨게임즈</corp_name><stock_code>225570</stock_code><corp_cls>K</corp_cls><report_nm>분기보고서 (2020.03)</report_nm><rcept_no>20200515001604</rcept_no><flr_nm>넥슨게임즈</flr_nm><rcept_dt>20200515</rcept_dt><rm></rm></list><list><corp_code>01

In [126]:
import xml.etree.ElementTree as ET

#tree = ET.parse(webpage) 이건 파일을 파싱하는거
reports = {}  # 보고서 명 - 보고서 코드 딕셔너리 변수 선언
for webpage in webpages:
  tree = ET.fromstring(webpage) # XML 문자열을 파싱하는 경우 이걸 사용

  root_element = ET.fromstring(webpage)
  #report_list = [] #보고서 리스트 변수 선언

  iter_element = root_element.iter(tag="list")

  for element in iter_element:
    report_name = element.find("report_nm").text
    # 보고서 이름 저장  XX보고서 (YYYY .MM)
    report_code = ''

    code = element.find("corp_code").text
    print(code)

    rept_nm, rept_dt = report_name.split(' ')
    rept_nm.strip()
    rept_dt = rept_dt[1:-1]
    report_date = (rept_dt.split('.'))
    # 1분기 / 3분기 보고서를 구분하기 위한 문자열 스플릿

    print(rept_nm ,end='-')
    print(report_date)

    if rept_nm == '분기보고서' :
      report_code = '11012' #반기 보고서 코드 할당
    elif rept_nm == '사업보고서' :
      report_code = '11011' #사업 보고서 코드 할당
    else :
      if report_date[1] == '03':
        report_code='11013' #1분기 보고서 코드 할당
      else :
        report_code='11014' #3분기 보고서 코드 할당

    report_date = tuple(report_date) # 보고서 발행 날짜를 튜플로 변환하여 딕셔너리에 추가
    reports[report_date] = report_code

reports = dict(sorted(reports.items(), key=lambda x: x[0]))
print(reports)


01096341
분기보고서-['2020', '09']
01096341
반기보고서-['2020', '06']
01096341
분기보고서-['2020', '03']
01096341
사업보고서-['2019', '12']
01096341
분기보고서-['2021', '09']
01096341
반기보고서-['2021', '06']
01096341
분기보고서-['2021', '03']
01096341
사업보고서-['2020', '12']
01096341
분기보고서-['2022', '09']
01096341
반기보고서-['2022', '06']
01096341
분기보고서-['2022', '03']
01096341
사업보고서-['2021', '12']
01096341
분기보고서-['2023', '09']
01096341
[기재정정]반기보고서-['2023', '06']
01096341
반기보고서-['2023', '06']
01096341
분기보고서-['2023', '03']
01096341
사업보고서-['2022', '12']
01096341
분기보고서-['2024', '09']
01096341
반기보고서-['2024', '06']
01096341
분기보고서-['2024', '03']
01096341
사업보고서-['2023', '12']
{('2019', '12'): '11011', ('2020', '03'): '11012', ('2020', '06'): '11014', ('2020', '09'): '11012', ('2020', '12'): '11011', ('2021', '03'): '11012', ('2021', '06'): '11014', ('2021', '09'): '11012', ('2021', '12'): '11011', ('2022', '03'): '11012', ('2022', '06'): '11014', ('2022', '09'): '11012', ('2022', '12'): '11011', ('2023', '03'): '11012', ('2023', '06'

In [127]:
import requests

bsns_year = ['2020', '2021', '2022', '2023', '2024'] # Changed to string list to match report_date tuple


requested_reports = []

for year, month in reports.keys():
  print(year, reports[(year, month)]) #사업연도, 보고서 코드 확인

  # Corrected URL format string
  url = 'https://opendart.fss.or.kr/api/fnlttSinglAcnt.xml?crtfc_key={}&corp_code={}&bsns_year={}&reprt_code={}'.format(crtfc_key, corp_code, year, reports[(year, month)])
  #url = 'https://opendart.fss.or.kr/api/fnlttSinglAcnt.xml?crtfc_key={}&corp_code={}&bsns_year={}&reprt_code={}'.format(crtfc_key, corp_code, year, reports[(year, month)])
  print('url ', url)

  resp = requests.get(url)

  requested_reports.append(resp.content.decode('UTF-8'))
  print(f"Response for {year} ({month}) with report code {reports[(year, month)]}:\n{requested_reports[-1]}\n") # Print each response



2019 11011
url  https://opendart.fss.or.kr/api/fnlttSinglAcnt.xml?crtfc_key=be1cf4dc3136fbd49a414efc1dd8141b30c6354f&corp_code=01096341&bsns_year=2019&reprt_code=11011
Response for 2019 (12) with report code 11011:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><result><status>000</status><message>정상</message><list><rcept_no>20200330002091</rcept_no><reprt_code>11011</reprt_code><bsns_year>2019</bsns_year><corp_code>01096341</corp_code><stock_code>225570</stock_code><fs_div>OFS</fs_div><fs_nm>재무제표</fs_nm><sj_div>BS</sj_div><sj_nm>재무상태표</sj_nm><account_nm>유동자산</account_nm><thstrm_nm>제 5 기</thstrm_nm><thstrm_dt>2019.12.31 현재</thstrm_dt><thstrm_amount>53,650,461,527</thstrm_amount><frmtrm_nm>제 4 기</frmtrm_nm><frmtrm_dt>2018.12.31 현재</frmtrm_dt><frmtrm_amount>22,159,210,178</frmtrm_amount><bfefrmtrm_nm>제 3 기</bfefrmtrm_nm><bfefrmtrm_dt>2017.12.31 현재</bfefrmtrm_dt><bfefrmtrm_amount>35,726,056,445</bfefrmtrm_amount><ord>2</ord><currency>KRW</currency></list><list><rcept_no>2020033000

In [147]:
import xml.etree.ElementTree as ET
import pandas as pd

all_report_data = []

for requested_report in requested_reports:
    tree = ET.fromstring(requested_report)
    root_element = ET.fromstring(requested_report)

    # Extract data from 'list' elements within each report's XML
    for item in root_element.findall("list"):
        data = {
            'rcept_no': item.findtext('rcept_no'),
            'bsns_year': item.findtext('bsns_year'), # Get from XML
            'reprt_code': item.findtext('reprt_code'), # Get from XML
            'account_nm': item.findtext('account_nm'),
            'fs_div': item.findtext('fs_div'),
            'sj_div': item.findtext('sj_div'),
            'thstrm_nm': item.findtext('thstrm_nm'),
            'thstrm_dt': item.findtext('thstrm_dt'),
            'thstrm_amount': item.findtext('thstrm_amount'),
            'frmtrm_nm': item.findtext('frmtrm_nm'),
            'frmtrm_dt': item.findtext('frmtrm_dt'),
            'frmtrm_amount': item.findtext('frmtrm_amount'),
            'currency': item.findtext('currency'),
        }
        all_report_data.append(data)


# Create a single DataFrame from all collected data
df = pd.DataFrame(all_report_data)

# Display the DataFrame
display(df)
df.to_csv('/content/NexonGames.csv')

,rcept_no,bsns_year,reprt_code,account_nm,fs_div,sj_div,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,currency
0,20200330002091,2019,11011,유동자산,OFS,BS,제 5 기,2019.12.31 현재,"53,650,461,527",제 4 기,2018.12.31 현재,"22,159,210,178",KRW
1,20200330002091,2019,11011,비유동자산,OFS,BS,제 5 기,2019.12.31 현재,"19,295,571,567",제 4 기,2018.12.31 현재,"7,256,714,003",KRW
2,20200330002091,2019,11011,자산총계,OFS,BS,제 5 기,2019.12.31 현재,"72,946,033,094",제 4 기,2018.12.31 현재,"29,415,924,181",KRW
3,20200330002091,2019,11011,유동부채,OFS,BS,제 5 기,2019.12.31 현재,"11,218,064,116",제 4 기,2018.12.31 현재,"4,510,336,944",KRW
4,20200330002091,2019,11011,비유동부채,OFS,BS,제 5 기,2019.12.31 현재,"23,577,196,403",제 4 기,2018.12.31 현재,"6,237,056,529",KRW
...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,20240808000287,2024,11012,영업이익,OFS,IS,제 10 기반기,2024.01.01 ~ 2024.06.30,"-9,077,795,912",제 9 기반기,2023.01.01 ~ 2023.06.30,"28,535,651",KRW
423,20240808000287,2024,11012,법인세차감전 순이익,OFS,IS,제 10 기반기,2024.01.01 ~ 2024.06.30,"-10,242,939,156",제 9 기반기,2023.01.01 ~ 2023.06.30,"-1,441,154,806",KRW
424,20240808000287,2024,11012,당기순이익(손실),OFS,IS,제 10 기반기,2024.01.01 ~ 2024.06.30,"-11,264,458,464",제 9 기반기,2023.01.01 ~ 2023.06.30,"-2,927,471,461",KRW
425,20240808000287,2024,11012,총포괄손익,OFS,IS,제 10 기반기,2024.01.01 ~ 2024.06.30,"-11,264,458,464",제 9 기반기,2023.01.01 ~ 2023.06.30,"-2,927,471,461",KRW


In [ ]:
import